In [1]:
!pip install -U spacy==2.3.5

  Using cached spacy-2.3.5-cp39-cp39-win_amd64.whl (9.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-md 3.0.0 requires spacy<3.1.0,>=3.0.0, but you have spacy 2.3.5 which is incompatible.
You should consider upgrading via the 'c:\users\lhustvei\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
  Using cached catalogue-1.0.0-py2.py3-none-any.whl (7.7 kB)
  Using cached srsly-1.0.5-cp39-cp39-win_amd64.whl (177 kB)
  Using cached thinc-7.4.5-cp39-cp39-win_amd64.whl (904 kB)
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.0
    Uninstalling srsly-2.4.0:
      Successfully uninstalled srsly-2.4.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.1
    Uninstalling catalogue-2.0.1:
      Successfully uninstalled catalogue-2.0.1
  Attempting uninstall: thi

In [2]:
import spacy
print(spacy.__version__)

2.3.5


In [3]:
# Had to run this to make something below work.
# This is a donwgrade of numpy to a version without a bug.
# This is a temp solution until a new version of numpy has solved the bug.
!pip install numpy==1.19.3 --user

You should consider upgrading via the 'c:\users\lhustvei\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [4]:
def is_float(n):
    try:
        support_float_with_norwegian_format = n.replace(',','.')
        float_n = float(support_float_with_norwegian_format)
    except ValueError:
        return False
    else:
        return True
    
# test
print(is_float('test'))
print(is_float('4'))
print(is_float('4.5'))
print(is_float('4,5'))

False
True
True
True


In [5]:
import spacy
#from spacy.lang.en import English
from spacy.lang.nb import Norwegian
from spacy.matcher import Matcher

# Blank model Norwegian().
# Does not have a POS tagger.
nlp = Norwegian()

#text = "(1) Fartøy med største lengde mellom 10,67 og 15 meter skal ha brannslokningsanlegg."
text = "(4) Fartøy med største lengde mindre enn 10,67 meter bygget etter 1. januar 1992 kan alternativt være utstyrt med brannslokningsanlegg i samsvar med retningslinjer fastsatt i kapittel Y14 i Nordisk Båt Standard (1990)."

doc = nlp(text)

matcher = Matcher(nlp.vocab)

#
# START - spaCy patterns
#

# LENGTH

length_pattern = [
    {"TEXT": {"REGEX": "[0-9]+[,]+[0-9]|[0-9]"}},
    {"LOWER": {"IN": ["meter"]}}]

matcher.add("LENGTH", None, length_pattern)

# LENGTH_PREFIX

length_prefix_pattern_1 = [
    {"LOWER": {"IN": ["mindre"]}},
    {"LOWER": {"IN": ["enn"]}}]

length_prefix_pattern_2 = [
    {"LOWER": {"IN": ["under"]}}]

matcher.add("LENGTH_PREFIX", None, length_prefix_pattern_1, length_prefix_pattern_2)

# WATER_VESSEL

water_vessel_pattern = [
    {"LOWER": {"IN": ["fartøy"]}}]

matcher.add("WATER_VESSEL", None, water_vessel_pattern)

#
# END - spaCy patterns
#

result = []

for match_id, token_start, token_end in matcher(doc):

    match_id_as_string = nlp.vocab.strings[match_id]

    final_token_start = token_start
    final_token_end = token_end

    #
    # Expand result if extra length detected in the words before in the line
    #
    if match_id_as_string == "LENGTH" and token_start > 0:
        prev_token_1 = doc[token_start-1].text
        if prev_token_1 in ("og", "til"):
            prev_token_2 = doc[token_start-2].text
            if is_float(prev_token_2):
                final_token_start = token_start-2

    #
    # convert token_span to char_span.
    # char_span is needed to display correctly withdisplacy.render().
    #
    span = doc[final_token_start:final_token_end]
    span_char_start = span[0].idx
    span_char_end = span[-1].idx + len(span[-1].text)

    # return result
    identified_entity = {'start': span_char_start, 'end': span_char_end, 'label': match_id_as_string}
    result.append(identified_entity)

# display result with spacy styling
display_this_result = {'text': text, 'ents': sorted(result, key=lambda x: x['start']), 'title': 0}
options = {"colors": {"LENGTH": "yellow", "LENGTH_PREFIX": "orange", "WATER_VESSEL": "aquamarine"}}
from spacy import displacy
displacy.render(display_this_result, style='ent', manual=True, options=options)